In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.memory import ConversationBufferMemory

chat_model = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ]
)

memory = ConversationBufferMemory(
    llm=chat_model,
    max_token_limit=600,
    memory_key="rag_chat_history",
    return_messages=True,
)

In [8]:
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.storage import LocalFileStore


file_name = "document.txt"

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader(f"./files/{file_name}")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)
vectorstore = FAISS.from_documents(docs, cached_embeddings)
retriever = vectorstore.as_retriever()

vectorstore.save_local(f"index/{file_name}")

In [ ]:
vectorstore = FAISS.load_local("index/document.txt", embeddings)
retriever = vectorstore.as_retriever()

In [12]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda


rag_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}""",
        ),
        MessagesPlaceholder(variable_name="rag_chat_history"),
        ("human", "{question}"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["rag_chat_history"]

rag_chain = {
    "context": retriever,
    "question": RunnablePassthrough(),
    "rag_chat_history": RunnableLambda(load_memory),
} | rag_prompt | chat_model

def invoke_rag_chain(question):
    result = rag_chain.invoke(question)
    memory.save_context(
        {"input": question},
        {"output": result.content}
    )


invoke_rag_chain("Is Aaronson guilty?")


According to the text, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.

In [14]:
invoke_rag_chain("What message did he write in the table?")

He wrote "2+2=5" on the table.

In [16]:
invoke_rag_chain("Who is Julia?")

Julia is a character in the text who is associated with the protagonist and plays a significant role in the story.

In [17]:
invoke_rag_chain("What I asked before?")

You asked who Julia is.

In [18]:
invoke_rag_chain("What I asked first?")

You asked if Aaronson is guilty.